In [ ]:
!pip install wandb

In [2]:
import pandas as pd
import numpy as np
from PIL import Image
import math
import random
import wandb

from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

import torchaudio
import torchaudio.transforms as T
from torchaudio.transforms import AmplitudeToDB, Vol

device = torch.device("cuda:0")

In [2]:
from BCResNet import MHAttKWS

In [3]:
wandb.login()

wandb: Currently logged in as: darya-dare. Use `wandb login --relogin` to force relogin


True

### Custom Dataset

In [3]:
class AudioDataset(Dataset):

    def __init__(self, path_speech, path_noise, eps_value=1e-6,\
                 mel_nframes=300, sample_rate=16000, n_fft=480,\
                 hop_length=160, n_mels=40):

        self.speech = pd.read_csv(path_speech)
        self.noise = pd.read_csv(path_noise)
        self.eps_value = eps_value
        self.mel_nframes = mel_nframes
        self.sample_rate = sample_rate
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.n_mels = n_mels
        self.mel_spectrogram = T.MelSpectrogram(sample_rate=self.sample_rate,
                                                n_fft=self.n_fft,
                                                hop_length=self.hop_length,
                                                center=True,
                                                pad_mode="reflect",
                                                power=2.0,
                                                norm='slaney',
                                                n_mels=self.n_mels)

    def __len__(self):

        return len(self.noise)

    def get_same_shape(self, waveform, w_type):

        melsp = torch.log(self.mel_spectrogram(waveform) + self.eps_value)
        has_mel_nframes = melsp.size(2) if melsp.size(2) < self.mel_nframes\
         else self.mel_nframes

        image_tens = torch.full((1, self.n_mels, self.mel_nframes), melsp.min())
        image_tens[0, :, :has_mel_nframes] = melsp[:, :, :has_mel_nframes]

        return image_tens/255

    def load_item(self, df, idx, w_type):

        sample_wav_file = df['filename'][idx]
        wave, _ = torchaudio.load(sample_wav_file)
        wave = wave.float()

        if w_type==1:
            wave_noise = wave.numpy()
            wave_noise_no_silence = [float(el) for el in wave_noise[0] if abs(el) >= 0.001]
            wave = torch.tensor([wave_noise_no_silence])

        wave_melsp = self.get_same_shape(wave, w_type)

        return wave_melsp

    def __getitem__(self, idx):

        wave_speech = self.load_item(self.speech, idx, w_type=0)

        luck = random.random()
        if luck > 0.5:
            wave_noise = self.load_item(self.noise, idx, w_type=1)

            wave_out = wave_speech + wave_noise
            label = 1

        else:
            wave_out = wave_speech
            label = 0

        return wave_out, torch.tensor(label)

In [5]:
audio_dataset_train = AudioDataset('./CleanSpeechTrain.csv',
                                   './NoiseTrain.csv')

audio_dataset_val = AudioDataset('./CleanSpeechVal.csv',
                                   './NoiseVal.csv')

audio_dataset_test = AudioDataset('./CleanSpeechTest.csv',
                                   './NoiseTest.csv')

### Data Loader

In [6]:
train_loader = DataLoader(dataset=audio_dataset_train,
                          batch_size=50, drop_last=True)

val_loader = DataLoader(dataset=audio_dataset_val,
                        batch_size=50, drop_last=True)

test_loader = DataLoader(dataset=audio_dataset_test,
                         batch_size=50, drop_last=True)

### Train functions

In [7]:
def train(model, train_loader, val_loader, loss,
          optimizer, scheduler, num_epochs):

    wandb.init(
        project="my-project",

        config={"learning_rate": 1e-2,
                "model": "MHAttKWS",
                'loss': 'BCEWithLogitsLoss',
                "optimizer": "SGD",
                'scheduler': 'StepLR',
                "epochs": 5}
    )

    loss_history = []
    train_history = []
    val_history = []

    for epoch in range(num_epochs):

        model.train()

        loss_accum = 0
        correct_samples = 0
        total_samples = 0

        with tqdm(total=len(train_loader),
                  desc=f'Epoch {epoch+1}',
                  leave=True) as pb:

            for i, (x, y) in enumerate(train_loader):

                x_gpu = x.to(device)
                y_gpu = y.to(device)

                prediction = model(x_gpu)
                value, indices = torch.max(prediction, 1)

                loss_value = loss(value, y_gpu.float())
                optimizer.zero_grad()
                loss_value.backward()
                optimizer.step()

                correct_samples += torch.sum(indices==y_gpu)
                total_samples += y.size(0)
                loss_accum += loss_value

                pb.update()
                pb.set_description( f'Epoch {epoch+1}:'
                                    f' Average loss: {loss_value:.3f},'
                                    f' Train accuracy:'
                                    f' {correct_samples/total_samples:.2f},'
                                    )

            ave_loss = loss_accum / epoch+1
            train_accuracy = float(correct_samples) / total_samples
            val_accuracy = compute_accuracy(model, val_loader)

            scheduler.step()

            loss_history.append(float(ave_loss))
            train_history.append(train_accuracy)
            val_history.append(val_accuracy)

            wandb.log({'Epoch': epoch+1, 'loss': ave_loss,
                       'train_acc': train_accuracy,
                       'val_acc': val_accuracy})

            pb.set_description( f'Epoch {epoch+1}:'
                                f' Average loss: {ave_loss:.3f},'
                                f' Train accuracy: {train_accuracy:.2f},'
                                f' Val accuracy: {val_accuracy:.2f}'
                                )

    wandb.finish()
    return loss_history, train_history, val_history


def compute_accuracy(model, loader):

    model.eval()

    correct_samples = 0
    total_samples = 0

    for i, (x, y) in enumerate(loader):

        x_gpu = x.to(device)
        y_gpu = y.to(device)

        prediction = model(x_gpu)
        indices = torch.argmax(prediction, 1)
        correct_samples += torch.sum(indices==y_gpu)
        total_samples += y_gpu.size(0)

    return float(correct_samples) / total_samples

### MHAttKWS

In [9]:
model_mhat = MHAttKWS(num_classes=2)
model_mhat.type(torch.FloatTensor)
model_mhat.to(device)

MHAttKWS(
  (cnn_extractor): Sequential(
    (0): Conv2d(1, 10, kernel_size=(5, 1), stride=(1, 1))
    (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(10, 1, kernel_size=(5, 1), stride=(1, 1))
    (4): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (rnn): LSTM(1, 128, num_layers=2, batch_first=True, bidirectional=True)
  (q_emb): Linear(in_features=256, out_features=1024, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (fc): Sequential(
    (0): Linear(in_features=1024, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=2, bias=True)
  )
)

In [10]:
loss = nn.BCEWithLogitsLoss().type(torch.FloatTensor)

optimizer = optim.SGD(model_mhat.parameters(),
                       lr=1e-2,
                       weight_decay=1e-4)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)

In [11]:
loss_history, train_history, val_history = train(model_mhat, train_loader,
                                                 val_loader, loss,
                                                 optimizer, scheduler,
                                                 num_epochs=1)

Epoch 1: Average loss: inf, Train accuracy: 0.48, Val accuracy: 0.50: 100%|██████████| 61/61 [09:13<00:00,  9.07s/it]


Epoch,▁
train_acc,▁
val_acc,▁
Epoch,1
loss,inf
train_acc,0.48
val_acc,0.502
